# Population Density

In [177]:
import clickhouse_connect
import polars as pl
import matplotlib.pyplot as plt
import numpy as np
import geopandas as gp
import pandas as pd

In [178]:
client = clickhouse_connect.get_client(host='hub.publichealthhq.xyz', port=18123, username='default', password='Password123!')

## Get Pre-computed Gini Coefficients

In [179]:
res = client.query('SELECT STATE_COUNTY_FIPS, GINI FROM cps_00004.county_gini')
gini_df = pl.from_dicts(res.named_results(), infer_schema_length=400)
gini_df = gini_df.to_pandas()

In [180]:
gini_df['STATE_COUNTY_FIPS'] = gini_df['STATE_COUNTY_FIPS'].astype(int)
# Get rid of extreme outliers
gini_filtered = gini_df[ (0 < gini_df['GINI']) & (gini_df['GINI'] < 100) ]

## Get Tax Data

In [181]:
res = client.query('SELECT COUNTYFIP, (sum(ADJUSTED_GROSS_INCOME) / sum(NUM_RETURNS)) as avg_agi FROM cps_00004.income_tax GROUP BY COUNTYFIP')
tax_df = pl.from_dicts(res.named_results(), infer_schema_length=400).to_pandas()
tax_df['STATE_COUNTY_FIPS'] = tax_df['COUNTYFIP']

In [182]:
tax_gini_filtered = pd.merge(gini_filtered, tax_df, on='STATE_COUNTY_FIPS', how='inner', validate='1:1')

## Get Geospatial Data

In [183]:
gdf = gp.read_file('counties.geojson')
gdf['STATE_COUNTY_FIPS'] = (gdf['STATE'].astype(str) + gdf['COUNTY'].astype(str)).astype(int)
gdf['geometry']

0       POLYGON ((-86.49677 32.34444, -86.71790 32.402...
1       POLYGON ((-86.57780 33.76532, -86.75914 33.840...
2       POLYGON ((-85.18413 32.87053, -85.12342 32.772...
3       POLYGON ((-86.51734 33.02057, -86.51596 32.929...
4       POLYGON ((-88.13999 34.58170, -88.13925 34.587...
                              ...                        
3216    MULTIPOLYGON (((-75.24227 38.02721, -75.29687 ...
3217    POLYGON ((-81.22510 37.23487, -81.20477 37.243...
3218    POLYGON ((-81.96830 37.53780, -81.92787 37.512...
3219    POLYGON ((-78.44332 37.07940, -78.49303 36.891...
3220    POLYGON ((-77.85180 37.35487, -77.85515 37.418...
Name: geometry, Length: 3221, dtype: geometry

In [184]:
gdf.to_crs(crs='EPSG:3857', inplace=True)
gdf_buf = gdf.copy()
gdf_buf['geometry'] = gdf.buffer(100)


In [185]:
joined_gini = pd.merge(gdf_buf, tax_gini_filtered, on=['STATE_COUNTY_FIPS'], how='inner', validate='1:1')

## Spatial Join
The single line below is really critical since it is performing the spatial join that will match up each county with the its neighboring counties.

The projected/selected spatial join results will be saved in Clickhouse to be used as a precomputed table. This will give us the ability to make some parts of the analysis interactive without having to wait for the expensive joins.

In [186]:
cross_joined = gp.sjoin(joined_gini, joined_gini, how='inner', predicate='intersects')
cross_joined = cross_joined[ cross_joined['STATE_COUNTY_FIPS_left'] !=  cross_joined['STATE_COUNTY_FIPS_right']]

In [187]:
selected = cross_joined[['STATE_left',  'STATE_COUNTY_FIPS_left', 'GINI_left', 'avg_agi_left','STATE_right', 'STATE_COUNTY_FIPS_right', 'GINI_right', 'avg_agi_right']]

### Save as CSV
We'll use this as a pre-computed table in the database since I never got turf to work for the spatial joins.

In [188]:
import csv
with open('neighboring_counties.csv', 'w') as f:
    writer = csv.DictWriter(f, fieldnames=['ROWNUM', 'STATE_COUNTY_FIPS_left', 'GINI_left', 'AVG_AGI_left', 'STATE_COUNTY_FIPS_right', 'GINI_right', 'AVG_AGI_right'])
    for (i, (_, row)) in enumerate(selected.iterrows()):
        writer.writerow({
            'ROWNUM': i,
            'STATE_COUNTY_FIPS_left': row['STATE_COUNTY_FIPS_left'],
            'GINI_left': row['GINI_left'],
            'AVG_AGI_left': row['avg_agi_left'],
            'STATE_COUNTY_FIPS_right': row['STATE_COUNTY_FIPS_right'],
            'GINI_right': row['GINI_right'],
            'AVG_AGI_right': row['avg_agi_right']
        })

## Group counties
Here we group the counties into four groups:

1. Poor counties neighbored only by poor counties
2. Rich counties neighbored only by rich counties
3. Poor counties neighbored by at least one rich county
4. Rich counties neighbored by at least one poor county

In [189]:
poor = selected['avg_agi_right'].quantile(0.10)
rich = selected['avg_agi_right'].quantile(0.90)


selected_pl = pl.from_pandas(selected)

# Just showing the same thing with polars
# poor_pl = selected_pl['avg_agi_left'].quantile(0.10)

print(f'poor: ${poor:.2f}')
print(f'rich: ${rich:.2f}')

poor_near_poor = selected_pl\
.filter(pl.col('avg_agi_left') <= poor )\
.group_by('STATE_COUNTY_FIPS_left')\
    .agg( avg_agi_right_max=pl.col('avg_agi_right').max() )\
.filter( pl.col('avg_agi_right_max') <= poor )

rich_near_rich = selected_pl\
.filter( pl.col('avg_agi_left') >= rich )\
.group_by('STATE_COUNTY_FIPS_left')\
    .agg( avg_agi_right_max=pl.col('avg_agi_right').max() )\
.filter( pl.col('avg_agi_right_max') >= rich )

poor_near_rich = selected_pl\
.filter(pl.col('avg_agi_left') <= poor )\
.group_by('STATE_COUNTY_FIPS_left')\
    .agg( avg_agi_right_max=pl.col('avg_agi_right').max() )\
.filter( pl.col('avg_agi_right_max') >= rich )

# Notice that this uses min in the agg, not max
rich_near_poor = selected_pl\
.filter(pl.col('avg_agi_left') >= rich )\
.group_by('STATE_COUNTY_FIPS_left')\
    .agg( avg_agi_right_max=pl.col('avg_agi_right').min() )\
.filter( pl.col('avg_agi_right_max') <= poor )

poor: $38848.62
rich: $66677.44


In [190]:
print(f'poor_near_poor: {len(poor_near_poor)}')
print(f'rich_near_rich: {len(rich_near_rich)}')
print(f'poor_near_rich: {len(poor_near_rich)}')
print(f'rich_near_poor: {len(rich_near_poor)}')

poor_near_poor: 9
rich_near_rich: 279
poor_near_rich: 30
rich_near_poor: 27


In [191]:
pnp_list = list(poor_near_poor['STATE_COUNTY_FIPS_left'])
rnr_list = list(rich_near_rich['STATE_COUNTY_FIPS_left'])
pnr_list = list(poor_near_rich['STATE_COUNTY_FIPS_left'])
rnp_list = list(rich_near_poor['STATE_COUNTY_FIPS_left'])

In [192]:
pnp_list

[13243, 48215, 48489, 28119, 51595, 48061, 29179, 21057, 21129]

In [193]:
pnp_fips = poor_near_poor.select(['STATE_COUNTY_FIPS_left'])
renamed_pnp = selected_pl\
.join(pnp_fips, on='STATE_COUNTY_FIPS_left', how='inner')\
.rename({
    'STATE_left': 'STATE_FIPS_focus',
    'STATE_COUNTY_FIPS_left': 'STATE_COUNTY_FIPS_focus',
    'GINI_left': 'GINI_focus',
    'avg_agi_left': 'AVG_AGI_focus',
    'STATE_right': 'STATE_FIPS_adj',
    'STATE_COUNTY_FIPS_right': 'STATE_COUNTY_FIPS_adj',
    'GINI_right': 'GINI_adj',
    'avg_agi_right': 'AVG_AGI_adj'
})

In [194]:
rnr_fips = rich_near_rich.select(['STATE_COUNTY_FIPS_left'])
renamed_rnr = selected_pl\
.join(rnr_fips, on='STATE_COUNTY_FIPS_left', how='inner')\
.rename({
    'STATE_left': 'STATE_FIPS_focus',
    'STATE_COUNTY_FIPS_left': 'STATE_COUNTY_FIPS_focus',
    'GINI_left': 'GINI_focus',
    'avg_agi_left': 'AVG_AGI_focus',
    'STATE_right': 'STATE_FIPS_adj',
    'STATE_COUNTY_FIPS_right': 'STATE_COUNTY_FIPS_adj',
    'GINI_right': 'GINI_adj',
    'avg_agi_right': 'AVG_AGI_adj'
})

In [195]:
pnr_fips = poor_near_poor.select(['STATE_COUNTY_FIPS_left'])
renamed_pnr = selected_pl\
.join(pnr_fips, on='STATE_COUNTY_FIPS_left', how='inner')\
.rename({
    'STATE_left': 'STATE_FIPS_focus',
    'STATE_COUNTY_FIPS_left': 'STATE_COUNTY_FIPS_focus',
    'GINI_left': 'GINI_focus',
    'avg_agi_left': 'AVG_AGI_focus',
    'STATE_right': 'STATE_FIPS_adj',
    'STATE_COUNTY_FIPS_right': 'STATE_COUNTY_FIPS_adj',
    'GINI_right': 'GINI_adj',
    'avg_agi_right': 'AVG_AGI_adj'
})

In [196]:
rnp_fips = poor_near_poor.select(['STATE_COUNTY_FIPS_left'])
renamed_rnp = selected_pl\
.join(rnp_fips, on='STATE_COUNTY_FIPS_left', how='inner')\
.rename({
    'STATE_left': 'STATE_FIPS_focus',
    'STATE_COUNTY_FIPS_left': 'STATE_COUNTY_FIPS_focus',
    'GINI_left': 'GINI_focus',
    'avg_agi_left': 'AVG_AGI_focus',
    'STATE_right': 'STATE_FIPS_adj',
    'STATE_COUNTY_FIPS_right': 'STATE_COUNTY_FIPS_adj',
    'GINI_right': 'GINI_adj',
    'avg_agi_right': 'AVG_AGI_adj'
})

In [197]:
res = client.query('''
        SELECT STATE_COUNTY_FIPS_left, GINI_left, AVG_AGI_left, STATE_COUNTY_FIPS_right, GINI_right, AVG_AGI_right 
        FROM cps_00004.neighboring_counties
    ''')
df = pl.from_dicts(res.named_results(), infer_schema_length=400)
df = df.select(
    pl.col('STATE_COUNTY_FIPS_left'), 
    pl.col('GINI_left'), 
    pl.col('AVG_AGI_left').cast(pl.Float64), 
    pl.col('STATE_COUNTY_FIPS_right'), 
    pl.col('GINI_right'), 
    pl.col('AVG_AGI_right').cast(pl.Float64)
)
poor = df['AVG_AGI_left'].quantile(0.10)
rich = df['AVG_AGI_left'].quantile(0.90)

poor_near_poor = df\
.filter(pl.col('AVG_AGI_left') <= poor )\
.group_by('STATE_COUNTY_FIPS_left')\
    .agg( AVG_AGI_right_max=pl.col('AVG_AGI_right').max() )\
.filter( pl.col('AVG_AGI_right_max') <= poor )\


rich_near_rich = df\
.filter( pl.col('AVG_AGI_left') >= rich )\
.group_by('STATE_COUNTY_FIPS_left')\
    .agg( AVG_AGI_right_max=pl.col('AVG_AGI_right').max() )\
.filter( pl.col('AVG_AGI_right_max') >= rich )\

poor_near_rich = df\
.filter(pl.col('AVG_AGI_left') <= poor )\
.group_by('STATE_COUNTY_FIPS_left')\
    .agg( AVG_AGI_right_max=pl.col('AVG_AGI_right').max() )\
.filter( pl.col('AVG_AGI_right_max') >= rich )\

# Notice that this uses min in the agg, not max
rich_near_poor = df\
.filter(pl.col('AVG_AGI_left') >= rich )\
.group_by('STATE_COUNTY_FIPS_left')\
    .agg( AVG_AGI_right_max=pl.col('AVG_AGI_right').min() )\
.filter( pl.col('AVG_AGI_right_max') <= poor ) \


pnp_fips = poor_near_poor.select(['STATE_COUNTY_FIPS_left'])
renamed_pnp = df\
.join(pnp_fips, on='STATE_COUNTY_FIPS_left', how='inner')\
.filter( pl.col('STATE_COUNTY_FIPS_left').ne(pl.col('STATE_COUNTY_FIPS_right')) )\
.rename({
    'STATE_COUNTY_FIPS_left': 'STATE_COUNTY_FIPS_focus',
    'GINI_left': 'GINI_focus',
    'AVG_AGI_left': 'AVG_AGI_focus',
    'STATE_COUNTY_FIPS_right': 'STATE_COUNTY_FIPS_adj',
    'GINI_right': 'GINI_adj',
    'AVG_AGI_right': 'AVG_AGI_adj'
})

rnr_fips = rich_near_rich.select(['STATE_COUNTY_FIPS_left'])
renamed_rnr = df\
.join(rnr_fips, on='STATE_COUNTY_FIPS_left', how='inner')\
.filter( pl.col('STATE_COUNTY_FIPS_left').ne(pl.col('STATE_COUNTY_FIPS_right')) )\
.rename({
    'STATE_COUNTY_FIPS_left': 'STATE_COUNTY_FIPS_focus',
    'GINI_left': 'GINI_focus',
    'AVG_AGI_left': 'AVG_AGI_focus',
    'STATE_COUNTY_FIPS_right': 'STATE_COUNTY_FIPS_adj',
    'GINI_right': 'GINI_adj',
    'AVG_AGI_right': 'AVG_AGI_adj'
})

pnr_fips = poor_near_rich.select(['STATE_COUNTY_FIPS_left'])
renamed_pnr = df\
.join(pnr_fips, on='STATE_COUNTY_FIPS_left', how='inner')\
.filter( pl.col('STATE_COUNTY_FIPS_left').ne(pl.col('STATE_COUNTY_FIPS_right')) )\
.rename({
    'STATE_COUNTY_FIPS_left': 'STATE_COUNTY_FIPS_focus',
    'GINI_left': 'GINI_focus',
    'AVG_AGI_left': 'AVG_AGI_focus',
    'STATE_COUNTY_FIPS_right': 'STATE_COUNTY_FIPS_adj',
    'GINI_right': 'GINI_adj',
    'AVG_AGI_right': 'AVG_AGI_adj'
})

rnp_fips = rich_near_poor.select(['STATE_COUNTY_FIPS_left'])
renamed_rnp = df\
.join(rnp_fips, on='STATE_COUNTY_FIPS_left', how='inner')\
.filter( pl.col('STATE_COUNTY_FIPS_left').ne(pl.col('STATE_COUNTY_FIPS_right')) )\
.rename({
    'STATE_COUNTY_FIPS_left': 'STATE_COUNTY_FIPS_focus',
    'GINI_left': 'GINI_focus',
    'AVG_AGI_left': 'AVG_AGI_focus',
    'STATE_COUNTY_FIPS_right': 'STATE_COUNTY_FIPS_adj',
    'GINI_right': 'GINI_adj',
    'AVG_AGI_right': 'AVG_AGI_adj'
})

In [198]:
renamed_rnp.join(renamed_pnr, left_on='STATE_COUNTY_FIPS_adj', right_on='STATE_COUNTY_FIPS_focus', how='inner')

STATE_COUNTY_FIPS_focus,GINI_focus,AVG_AGI_focus,STATE_COUNTY_FIPS_adj,GINI_adj,AVG_AGI_adj,GINI_focus_right,AVG_AGI_focus_right,STATE_COUNTY_FIPS_adj_right,GINI_adj_right,AVG_AGI_adj_right
i64,f64,f64,i64,f64,f64,f64,f64,i64,f64,f64
13145,0.820368,68696.014,1017,0.815764,35732.164,0.815764,35732.164,1081,0.895927,54672.451
13145,0.820368,68696.014,1017,0.815764,35732.164,0.815764,35732.164,1123,0.816752,45983.304
13145,0.820368,68696.014,1017,0.815764,35732.164,0.815764,35732.164,1111,0.792802,41133.896
13145,0.820368,68696.014,1017,0.815764,35732.164,0.815764,35732.164,13145,0.820368,68696.014
13145,0.820368,68696.014,1017,0.815764,35732.164,0.815764,35732.164,13285,0.915647,45676.001
2122,0.859251,67674.62,2050,0.843077,37838.746,0.843077,37838.746,2164,0.551697,34755.405
2170,0.875339,68762.228,2050,0.843077,37838.746,0.843077,37838.746,2164,0.551697,34755.405
2122,0.859251,67674.62,2050,0.843077,37838.746,0.843077,37838.746,2070,0.643346,44266.81
2170,0.875339,68762.228,2050,0.843077,37838.746,0.843077,37838.746,2070,0.643346,44266.81
